In [1]:
from keras.layers import *
from keras.models import Model

import numpy as np
from tensorflow_model_optimization.quantization.keras import vitis_quantize
import os
import random 

# def get_total_number_of_neurons(model, include_output_layer):
#     '''
#     Args:
#         model: Keras model
#         include_output_layer: A boolean parameter. Whether or not to include output layer's neurons into the calculation

#     Returns: number of neurons in the given model
#     '''
#     num_layers = len(model.layers)
#     total_num_of_neurons = 0
#     for layer_index in range(num_layers):
#         layer = model.layers[layer_index]
#         # since we multiply layer output dimensions, initial value is set to 1.
#         num_neurons_in_layer = 1
#         for i in range(1, len(layer.output.shape)):
#             try:
#                 # when it is a valid layer to count neurons, an output dimension of the layer can be convertible to int.
#                 num_neurons_in_layer *= int(layer.output.shape[i])
#             except Exception:
#                 # if the output dimension of layer cannot be convertible to int,
#                 # just pass that layer since it is not a valid layer to count neurons
#                 pass
#         # if num_neurons_in_layer is not still 1, it means we have a valid layer to count neurons
#         if not num_neurons_in_layer == 1:
#             # when it is an output layer
#             if layer_index == (num_layers - 1):
#                 if include_output_layer:
#                     total_num_of_neurons += num_neurons_in_layer
#             else:  # when it is not an output layer
#                 total_num_of_neurons += num_neurons_in_layer
#     return total_num_of_neurons

shape_range =range(160,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    
    inputs = tf.keras.Input(shape=(shape_x,shape_y,1))
    for step in range(5,11):
        conv1_kernel_num = random.randrange(32,64,2)
        x = tf.keras.layers.Conv2D(conv1_kernel_num, (3,3), activation='relu')(inputs)
        x = tf.keras.layers.MaxPooling2D((2,2))(x)
        conv2_kernel_num = random.randrange(conv1_kernel_num,64,2)
        x = tf.keras.layers.Conv2D(conv2_kernel_num, (3,3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D((2,2))(x)
        conv3_kernel_num = random.randrange(conv2_kernel_num,64,2)
        x = tf.keras.layers.Conv2D(conv3_kernel_num, (3,3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D((2,2))(x)
        conv4_kernel_num = random.randrange(conv3_kernel_num,128,2)
        x = tf.keras.layers.Conv2D(conv4_kernel_num, (3,3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D((2,2))(x)
        conv5_kernel_num = random.randrange(conv4_kernel_num,180,2)
        x = tf.keras.layers.Conv2D(conv4_kernel_num, (3,3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D((2,2))(x)
        x = tf.keras.layers.Flatten()(x)
        Dense1_neuron_num = random.randrange(1024,2048,32)
        x = tf.keras.layers.Dense(Dense1_neuron_num, activation='relu')(x)
        Dense2_neuron_num = random.randrange(512,Dense1_neuron_num+1,32)
        x = tf.keras.layers.Dense(Dense2_neuron_num, activation='relu')(x)
        Dense3_neuron_num = random.randrange(256,Dense2_neuron_num+1,32)
        x = tf.keras.layers.Dense(Dense3_neuron_num, activation='relu')(x)
        outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

        model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_classifier')
        model.summary()
        

        model.save('./synthetic_h5/synthetic_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))

        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
#         quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        quantized_model.save('quantized.h5')
        !vai_c_tensorflow2 \
        --model ./quantized.h5 \
        --arch ./arch.json \
        --output_dir ./xmodel_synthetic \
        --net_name deploy
        os.rename("./xmodel_synthetic/deploy.xmodel","./xmodel_synthetic/synthetic_{shape_x}_{shape_y}_{step}.xmodel".format(shape_x=shape_x,shape_y=shape_y,step=step))



2022-05-16 02:33:33.373599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-05-16 02:33:33.373629: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "mnist_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 158, 158, 38)      380       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 38)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 77, 38)        13034     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 38)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 56)        19208     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 56)   

2022-05-16 02:33:34.874883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-05-16 02:33:34.874914: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-16 02:33:34.874931: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-05-16 02:33:34.875073: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 1s 130ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...


2022-05-16 02:33:40.932193: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


...

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 1s 166ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 6s 6s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 225ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 6s 6s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 307ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 6s 6s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 286ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 6s 6s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 252ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 5s 5s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :